In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.4.2


In [2]:
!python -V

Python 3.10.13


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [21]:
#!pip install pyarrow
#!pip install prefect
#!pip install mlflow

  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached entrypoints-0.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached graphene-3.3-py2.py3-none-any.whl.metadata (7.7 kB)
  Using cached importlib_metadata-7.1.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl.metadata (559 bytes)
  Using cached sqlparse-0.5.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached gunicorn-22.0.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached werkzeug-3.0.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached graphql_core-3.2.3-py3-none-any.whl.metadata (10 kB)
  Using cached graphql_relay-3.2.0-py3-none-any.whl.metadata (12 kB)
  Using cached aniso8601-9.0.1-py2.py3-none-any.whl.metadata (23 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━

In [17]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-05.parquet

--2024-06-17 22:07:12--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-05.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.32.192.124, 13.32.192.190, 13.32.192.2, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.32.192.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58654627 (56M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2023-05.parquet’

yellow_tripdata_202 100%[===================>]  55.94M  85.0MB/s    in 0.7s    

2024-06-17 22:07:13 (85.0 MB/s) - ‘yellow_tripdata_2023-05.parquet’ saved [58654627/58654627]



In [18]:
df = read_data('/workspaces/MLOps-Zoomcamp/04-deployment/yellow_tripdata_2023-05.parquet')

In [19]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [11]:
year = 2023  #
month = 4
df_predicted = pd.DataFrame()
df_predicted['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_predicted['duration'] = y_pred

In [12]:
df_predicted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199715 entries, 0 to 3199714
Data columns (total 2 columns):
 #   Column    Dtype  
---  ------    -----  
 0   ride_id   object 
 1   duration  float64
dtypes: float64(1), object(1)
memory usage: 48.8+ MB


In [13]:
df_predicted.describe()

,duration
count,3.199715e+06
mean,1.429228e+01
std,6.353998e+00
min,-1.632884e+01
25%,1.097487e+01
50%,1.198622e+01
75%,1.426810e+01
max,7.004772e+01


In [14]:
df_predicted.head()

,ride_id,duration
0,2023/04_0,16.131302
1,2023/04_1,32.264816
2,2023/04_2,12.255573
3,2023/04_3,12.133617
4,2023/04_4,13.071997


In [15]:
df_predicted.to_parquet(
    '/workspaces/MLOps-Zoomcamp/04-deployment/yellow_trip_pred_2023-04.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

In [16]:
import os
archivo = '/workspaces/MLOps-Zoomcamp/04-deployment/yellow_trip_pred_2023-04.parquet'
tamaño_archivo = os.path.getsize(archivo)
tamaño_kb = tamaño_archivo / 1024
tamaño_mb = tamaño_kb / 1024

print(f"Tamaño del archivo: {tamaño_archivo} bytes")
print(f"Tamaño del archivo: {tamaño_kb:.2f} KB")
print(f"Tamaño del archivo: {tamaño_mb:.2f} MB")


Tamaño del archivo: 66211511 bytes
Tamaño del archivo: 64659.68 KB
Tamaño del archivo: 63.14 MB
